# Imports

In [1]:
import numpy as np    # linear algebra 
import pandas as pd   # Reading in and looking at data

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        



In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from keras.models import Sequential
from keras.layers import Conv2D, Dropout, BatchNormalization, Flatten, Dense, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Reading in data

In [3]:
df = pd.read_csv('../input/age-gender-and-ethnicity-face-data-csv/age_gender.csv')

In [4]:
df.head()

In [5]:
df.describe().T

In [6]:
plt.figure(figsize=(12, 8))
sns.distplot(df['age']);

In [7]:
sns.countplot(df['ethnicity']);

In [8]:
sns.countplot(df['gender']);


In [9]:
df['pixels'] = df['pixels'].map(lambda x: np.array(x.split(' '), dtype=np.float32).reshape(48, 48))

In [10]:
# Plotting some pictures

fig, axes = plt.subplots(1, 5, figsize=(20, 10))


for i in range(5):
    random_face = np.random.choice(len(df))
    age = df['age'][random_face]
    ethnicity = df['ethnicity'][random_face]
    gender = df['gender'][random_face]
    
    axes[i].set_title('Age: {0}, Ethnicity: {1}, Sex: {2}'.format(age, ethnicity, gender ))
    axes[i].imshow(df['pixels'][random_face])
    axes[i].axis('off')

# Age prediction

In [11]:
X = np.array(df['pixels'].to_list())
X_reshaped = X.reshape(X.shape[0], 48, 48, 1)

In [12]:
if np.max(X_reshaped) > 1: X_reshaped = X_reshaped / 255

In [13]:
input_shape = X_reshaped.shape[1:]

In [14]:
y = df['age'].values

In [15]:
# Train Test Split

X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

In [16]:
age_model = Sequential()
# For the activation function we'll be using 
# retified linear unit(relu) --> In a sense we have data that 
# could correlate with one another 

# Convolutional layer 1
age_model.add(Conv2D(64, kernel_size=(3,3), input_shape=input_shape, activation= 'relu'))
age_model.add(MaxPooling2D(pool_size=(2,2)))
age_model.add(BatchNormalization())


# Convolutional layer 2
age_model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
age_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
age_model.add(MaxPooling2D(pool_size=(2,2)))
age_model.add(Dropout(0.3))
age_model.add(BatchNormalization())

# Convolutional layer 3
age_model.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
age_model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
age_model.add(MaxPooling2D(pool_size=(2,2)))
age_model.add(Dropout(0.3))
age_model.add(BatchNormalization())


age_model.add(Flatten())
# Dense layer 1
age_model.add(Dense(128, activation='relu'))
age_model.add(Dropout(0.4))
# Dense layer 2
age_model.add(Dense(1))

# loss function 
age_model.compile(optimizer='adam', loss='mse')

age_model.summary()

In [17]:
n_epochs = 50  # epochs -> number of times model looks at 
               #the information
batch_size = 64 # batch_size -> a number of samples 
                #processed before the model is updated

In [18]:
# Callbacks for age model

callbacks = [EarlyStopping(patience=4, monitor='val_loss', mode='min'),
            ReduceLROnPlateau(patience=2, verbose=1)]

In [19]:
history = age_model.fit(X_train, y_train,
                       epochs=n_epochs,
                       batch_size=batch_size,
                       callbacks=callbacks,
                       validation_data=(X_test, y_test))

In [20]:
pd.DataFrame(history.history).plot();

In [21]:
valid_score = age_model.evaluate(X_test, y_test, verbose=1)

In [22]:
y_pred = age_model.predict(X_test)

In [23]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [24]:
print('MAE: {}'.format(mean_absolute_error(y_test, y_pred)))

# Gender Prediction

In [25]:
y = df['gender'].values
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

In [26]:
gender_model = Sequential()

# Convolutional layer 1
gender_model.add(Conv2D(64, kernel_size=(3,3), input_shape=input_shape, activation='relu'))
gender_model.add(MaxPooling2D(pool_size=(2,2)))
gender_model.add(BatchNormalization())

# Convolutional layer 2
gender_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
gender_model.add(MaxPooling2D(pool_size=(2,2)))
gender_model.add(Dropout(0.4))
gender_model.add(BatchNormalization())

# Convolutional layer 3
gender_model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
gender_model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
gender_model.add(MaxPooling2D(pool_size=(2,2)))
gender_model.add(Dropout(0.5))
gender_model.add(BatchNormalization())


gender_model.add(Flatten())

# Dense layer 1
gender_model.add(Dense(64, activation='relu'))
gender_model.add(Dropout(0.4))

# Dense layer 2
# For binary classification --> sigmoid activation function
gender_model.add(Dense(1, activation='sigmoid'))

# loss function is binary_crossentropy
gender_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

gender_model.summary()

In [27]:
# Callbacks for gender model 
callbacks = [EarlyStopping(patience=4, monitor='val_accuracy', mode='max'),
            ReduceLROnPlateau(patience=2, verbose=1)]

In [28]:
history = gender_model.fit(X_train, y_train,
                          epochs=n_epochs,
                          batch_size=batch_size,
                          callbacks=callbacks,
                          validation_data=(X_test, y_test))

In [29]:
valid_score = gender_model.evaluate(X_test, y_test, verbose=1)
print('Acc for test set: {}'.format(valid_score[1]))

In [30]:
y_pred = gender_model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

In [31]:
print(classification_report(y_test, y_pred))

In [34]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, cmap='Greens', cbar=False, annot=True, fmt='d');

In [35]:
y_pred = y_pred.reshape(y_pred.shape[0])

In [36]:
y = df['ethnicity'].values
y = to_categorical(y, 5)
y

# Ethnicity prediction

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=38)


In [38]:
eth_model = Sequential()


eth_model.add(Conv2D(64, kernel_size=(3,3), input_shape=input_shape, activation='relu'))
eth_model.add(MaxPooling2D(pool_size=(2,2)))
eth_model.add(BatchNormalization())

eth_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
eth_model.add(MaxPooling2D(pool_size=(2,2)))
eth_model.add(BatchNormalization())

eth_model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
eth_model.add(MaxPooling2D(pool_size=(2,2)))
eth_model.add(Dropout(0.5))
eth_model.add(BatchNormalization())

eth_model.add(Flatten())
eth_model.add(Dense(128, activation='relu'))
eth_model.add(Dropout(0.5))
eth_model.add(Dense(5, activation='softmax')) 

eth_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

eth_model.summary


In [39]:
history = eth_model.fit(X_train, y_train, epochs=n_epochs,
                       batch_size=batch_size,
                       callbacks=callbacks,
                       validation_data=(X_test, y_test))

In [40]:
valid_score = eth_model.evaluate(X_test, y_test, verbose=1)

In [41]:
predictions = (eth_model.predict(X_test) > 0.5).astype('int32')